<a href="https://colab.research.google.com/github/YoonJiHwan98/dacon_code/blob/main/gAs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

In [ ]:
!git clone https://github.com/YoonJiHwan98/TIL.git

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

plt.rc('font', family='NanumBarunGothic') 

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/TAVE 머신러닝 스터디/Gas/한국가스공사_시간별 공급량_20181231.csv',encoding = 'cp949')
test = pd.read_csv('/content/drive/MyDrive/TAVE 머신러닝 스터디/Gas/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/TAVE 머신러닝 스터디/Gas/sample_submission.csv')

In [ ]:
test

In [ ]:
data

In [ ]:
#data['연월일시'] = 
data['시간'] = data['시간'].astype(str)

In [ ]:
data['시간'] = data['시간'].map({'1' : '01','2' : '02','3' : '03','4' : '04','5' : '05','6' : '06','7' : '07','8' : '08','9' : '09','10' : '10','11' : '11','12' : '12','13' : '13','14' : '14','15' : '15','16' : '16','17' : '17','18' : '18','19' : '19','20' : '20','21' : '21','22' : '22','23' : '23','24' : '00'})

In [ ]:
data['시간'] = data['시간'].astype(str)

In [ ]:
data['연월일시'] = data['연월일'] + '-' + data['시간']
data

In [ ]:
data1 = data.copy()

In [ ]:
data1['연월일'] = data1['연월일'].astype(str)
data1['연월일시'] = data1['연월일'] + ' ' + data1['시간']

In [ ]:
data1['연월일시'] = data1['연월일시'].apply(lambda x : pd.to_datetime(x.split()[0]) + datetime.timedelta(days=1) if x.split()[1] == '24' else x)
data1['연월일시'] = pd.to_datetime(data1['연월일시'])
data1

In [ ]:
data1

In [ ]:
df_A = data[data['구분'] == "A"]

https://dacon.io/codeshare/2326

https://dacon.io/codeshare/2327?page&dtype&ptype

In [ ]:
#df['time'].apply(lambda x: pd.to_datetime(x.split()[0]) + timedelta)

In [ ]:
#pred_df = forecast.iloc[19008:]
#pred_df['DateTime'] = pred_df['ds'].apply(lambda x: datetime.datetime.strftime(x,'%Y-%m-%d'))

In [ ]:
#pred_df.groupby('DateTime')['yhat'].mean().reset_index(name = 'pred').head()

In [ ]:
data1

In [ ]:
#data['year'] = data['연월일'].dt.year
#data['month'] = data['연월일'].dt.month
#data['day'] = data['연월일'].dt.day
#data['weekday'] = data['연월일'].dt.weekday

In [ ]:
data1['구분'].unique()

In [ ]:
df_a = data1[data1['구분'] == 'A']
df_b = data1[data1['구분'] == 'B']
df_c = data1[data1['구분'] == 'C']
df_d = data1[data1['구분'] == 'D']
df_g = data1[data1['구분'] == 'G']
df_h = data1[data1['구분'] == 'H']
df_e = data1[data1['구분'] == 'E']


In [ ]:
sns.lineplot(data=df_h , x = '연월일시',y = '공급량')
plt.show()

In [ ]:
sns.lineplot(data=df_e , x = '연월일시',y = '공급량')
plt.show()

In [ ]:
df_h.loc[237216,'공급량'] =None
df_h.loc[237217,'공급량'] =None
df_h.loc[239116,'공급량']= None
df_h.loc[303299,'공급량']= None
df_h.loc[303300,'공급량']= None

# 아니면 람다 함수
#data1['연월일시'] = 
#df_h['공급량'].apply(lambda x : None if x.split()[1] == '24' else x)

In [ ]:
df_h.fillna(method='ffill',inplace = True)

In [ ]:
#sns.lineplot(data=df_h , x = '연월일시',y = '공급량')
plt.show()

In [ ]:
df_a[['ds','y']] = df_a[['연월일시','공급량']]
df_b[['ds','y']] = df_b[['연월일시','공급량']]
df_c[['ds','y']] = df_c[['연월일시','공급량']]
df_d[['ds','y']] = df_d[['연월일시','공급량']]
df_e[['ds','y']] = df_e[['연월일시','공급량']]
df_h[['ds','y']] = df_h[['연월일시','공급량']]
df_g[['ds','y']] = df_g[['연월일시','공급량']]

In [ ]:
train_a = df_a[['ds','y']]
train_b = df_b[['ds','y']]
train_c = df_c[['ds','y']]
train_d = df_d[['ds','y']]
train_e = df_e[['ds','y']]
train_h = df_h[['ds','y']]
train_g = df_g[['ds','y']]

In [ ]:
!pip install pystan==2.19.1.1
!pip install prophet
!pip install statsmodels==0.11.1
!pip install fbprophet==0.7.1

In [ ]:
from fbprophet import Prophet
model_a =Prophet() 
model_a.fit(train_a) 
future_A = model_a.make_future_dataframe(periods=2161, freq='H') 
forecast_A = model_a.predict(future_A)

In [ ]:
model_b =Prophet() 
model_b.fit(train_b) 
future_b = model_b.make_future_dataframe(periods=2161, freq='H') 
forecast_b = model_a.predict(future_b)

In [ ]:
model_c = Prophet() 
model_c.fit(train_c) 
future_c = model_c.make_future_dataframe(periods=2161, freq='H') 
forecast_c = model_c.predict(future_A)

In [ ]:
model_d =Prophet() 
model_d.fit(train_d) 
future_d = model_d.make_future_dataframe(periods=2161, freq='H') 
forecast_d = model_d.predict(future_d)

In [ ]:
model_e =Prophet() 
model_e.fit(train_e) 
future_e = model_e.make_future_dataframe(periods=2161, freq='H') 
forecast_e = model_e.predict(future_A)

In [ ]:
model_h =Prophet() 
model_h.fit(train_h) 
future_h = model_h.make_future_dataframe(periods=2161, freq='H') 
forecast_h = model_h.predict(future_A)

In [ ]:
model_g =Prophet() 
model_g.fit(train_g) 
future_g = model_g.make_future_dataframe(periods=2161, freq='H') 
forecast_g = model_g.predict(future_A)

In [ ]:
 pred_a = forecast_A.loc[52585:,'yhat']
 pred_b = forecast_b.loc[52585:,'yhat']
 pred_c = forecast_c.loc[52585:,'yhat']
 pred_d = forecast_d.loc[52585:,'yhat']
 pred_e = forecast_e.loc[52585:,'yhat']
 pred_h = forecast_h.loc[52585:,'yhat']
 pred_g = forecast_g.loc[52585:,'yhat']

In [ ]:
#testA = test[test['일자|시간|구분'].str.contains('A')]
#testB = test[test['일자|시간|구분'].str.contains('B')]
#testC = test[test['일자|시간|구분'].str.contains('C')]
#testD = test[test['일자|시간|구분'].str.contains('D')]
#testE = test[test['일자|시간|구분'].str.contains('E')]
#testG = test[test['일자|시간|구분'].str.contains('G')]
#testH = test[test['일자|시간|구분'].str.contains('H')]

In [ ]:
#pred_a = pred_a.reset_index()
pred_a['yhat']

In [ ]:
pred_a

In [ ]:
#total = 
total = pd.concat([pred_a,pred_b,pred_c,pred_d,pred_e,pred_h,pred_g])

In [ ]:
#total.to_frame().reset_index()['yhat']

In [ ]:
submission['공급량'] = total.to_frame().reset_index()['yhat']
submission

In [ ]:
submission.head(30)

In [ ]:
data['구분'] = data['구분'].astype('category')
data['weekday'] = data['weekday'].astype('category')
data['구분'] = data['구분'].map({'A' : '1','B' : '2','C' : '3','D' : '4','E' : '5','G' : '6','H' : '7'})

In [ ]:
submission.to_csv('/content/drive/MyDrive/TAVE 머신러닝 스터디/Gas/211105_1.csv', index = False,encoding = 'cp949')

In [ ]:
total.to_frame().reset_index()

In [ ]:
submission